In [96]:
import os
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### Cap maximums

In [151]:
data_cap = {'Year': [2024],
            'Cap Maximum': [142.00]}
cap = pd.DataFrame(data_cap)

cap1 = pd.read_html('https://www.spotrac.com/nba/cba/')[0]
cap1 = cap1[['Year','Cap Maximum']]
cap1['Cap Maximum'] = (cap1['Cap Maximum'].str.replace('$','').str.replace(',','')).astype(float)/1000000

cap = pd.concat([cap, cap1], ignore_index=True)
cap = cap.astype({'Year': 'int32'})
cap = cap.rename(columns={'Year': 'Season'})
cap.to_csv('../data/raw/cap.csv', index=False)
cap.head(15)

,Season,Cap Maximum
0,2024,142.00
1,2023,136.02
2,2022,123.66
3,2021,112.41
4,2020,109.14
5,2019,109.14
6,2018,101.87
7,2017,99.09
8,2016,94.14
9,2015,70.00


### Data cleanup

In [98]:
from unidecode import unidecode

def getPlayerStats():
    df_stats = pd.DataFrame()

    for year in range(2010, 2024):
        file_regular = os.path.join('..', 'data', 'raw', 'stats', f'{year} Regular.csv')
        file_playoff = os.path.join('..', 'data', 'raw', 'stats', f'{year} Playoff.csv')
        
        df_regular = pd.read_csv(file_regular)
        df_playoff = pd.read_csv(file_playoff)

        # Concat the two dataframes
        df = pd.concat([df_regular, df_playoff], ignore_index=True)
        df['Season'] = year
        
        # Normalizing names
        df['Player'] = df['Player'].str.replace('.','').str.replace('*','')
        df['Player'] = df['Player'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
        df['Player'] = df['Player'].apply(lambda x: x.replace(' Jr', '') if x.endswith(' Jr') and x not in ['Jaren Jackson Jr', 'Tim Hardaway Jr', 'Gary Trent Jr', 'Larry Nance Jr',
                                                                                                                 'Duane Washington Jr', 'Scottie Pippen Jr', 'Vince Williams Jr', 'Ron Harper Jr'] else x)
        df['Player'] = df['Player'].apply(unidecode)

        # Aggregate the stats, summing the regular season and playoff stats by player name
        df = df.groupby('Player').agg({
            'Pos': 'last', 'Age': 'last',
            'G': 'sum', 'GS': 'sum', 'MP': 'sum', 
            'FG': 'sum', 'FGA': 'sum', 
            '3P': 'sum', '3PA': 'sum', 
            '2P': 'sum', '2PA': 'sum', 
            'FT': 'sum', 'FTA': 'sum', 
            'ORB': 'sum', 'DRB': 'sum', 'TRB': 'sum', 
            'AST': 'sum', 'STL': 'sum', 'BLK': 'sum', 
            'TOV': 'sum', 'PF': 'sum', 'PTS': 'sum'})
        
        # Re-calculate the percentages
        df['FG%'] = (df['FG'] / df['FGA']).round(3)
        df['3P%'] = (df['3P'] / df['3PA']).round(3)
        df['2P%'] = (df['2P'] / df['2PA']).round(3)
        df['eFG%'] = ((df['FG'] + 0.5 * df['3P']) / df['FGA']).round(3)
        df['FT%'] = (df['FT'] / df['FTA']).round(3)

        df['Season'] = year
        df_stats = pd.concat([df_stats, df], ignore_index=False)
        
    df_stats = df_stats.reset_index()
    return df_stats
        


In [99]:
stats = getPlayerStats()
print(f'Number of rows: {stats.shape[0]}')
stats.head()

Number of rows: 7067


,Player,Pos,Age,G,GS,MP,FG,FGA,3P,3PA,2P,2PA,FT,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,FG%,3P%,2P%,eFG%,FT%,Season
0,AJ Price,PG,23,56,2,15.40,2.60,6.30,1.10,3.10,1.50,3.20,1.10,1.30,0.20,1.40,1.60,1.90,0.60,0.10,1.10,0.90,7.30,0.41,0.35,0.47,0.50,0.85,2010
1,Aaron Brooks,PG,25,82,82,35.60,7.00,16.20,2.50,6.40,4.50,9.80,3.00,3.60,0.70,2.00,2.60,5.30,0.80,0.20,2.80,2.40,19.60,0.43,0.39,0.46,0.51,0.83,2010
2,Aaron Gray,C,25,64,0,26.90,3.70,7.70,0.00,0.00,3.70,7.70,1.70,2.70,3.70,5.40,9.10,1.80,0.70,0.80,1.40,4.80,9.20,0.48,NaN,0.48,0.48,0.63,2010
3,Acie Law,PG,25,52,2,37.20,5.50,11.30,0.70,2.30,4.80,9.00,6.30,8.00,0.40,1.90,2.40,4.00,2.00,0.10,2.40,2.60,17.80,0.49,0.30,0.53,0.52,0.79,2010
4,Adam Morrison,SF,25,33,0,14.30,3.00,7.20,0.20,1.70,2.90,5.60,0.20,0.30,1.30,2.20,3.50,1.10,0.10,0.10,0.30,1.50,6.40,0.42,0.12,0.52,0.43,0.67,2010


In [100]:
def getPlayerSalaries():
    df_salaries = pd.DataFrame()

    for year in range(2010, 2024):
        file = os.path.join('..', 'data', 'raw', 'salaries', f'{year}.csv')
        df = pd.read_csv(file)

        # Normalizing names
        df['Player'] = df['Player'].str.replace('.','').str.replace('*','')
        df['Player'] = df['Player'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
        df['Player'] = df['Player'].apply(lambda x: x.replace(' Jr', '') if x.endswith(' Jr') and x not in ['Jaren Jackson Jr', 'Tim Hardaway Jr', 'Gary Trent Jr', 'Larry Nance Jr',
                                                                                                                 'Duane Washington Jr', 'Scottie Pippen Jr', 'Vince Williams Jr', 'Ron Harper Jr'] else x)
        df['Player'] = df['Player'].apply(unidecode)

        df['Salary'] = df['Salary'].fillna(-1000000)
        df['Salary'] = df['Salary']/1000000

        df['Season'] = year
        df_salaries = pd.concat([df_salaries, df], ignore_index=False)
        
    df_salaries = df_salaries.reset_index(drop=True)
    return df_salaries

In [101]:
salaries = getPlayerSalaries()
print(f'Number of rows: {salaries.shape[0]}')
salaries.head()

Number of rows: 8386


,Player,Salary,Season
0,Tracy McGrady,23.24,2010
1,Kobe Bryant,23.03,2010
2,Jermaine O'Neal,23.02,2010
3,Tim Duncan,22.18,2010
4,Shaquille O'Neal,20.00,2010


In [102]:
def get2kOverall():
    df_overall = pd.DataFrame()

    for year in range(10, 24):
        file = os.path.join('..', 'data', 'raw', '2k overall', f'2k{year}.csv')
        df = pd.read_csv(file)

        # Normalizing names
        df['Player'] = df['Player'].str.replace('.','').str.replace('*','')
        df['Player'] = df['Player'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
        df['Player'] = df['Player'].apply(lambda x: x.replace(' Jr', '') if x.endswith(' Jr') and x not in ['Jaren Jackson Jr', 'Tim Hardaway Jr', 'Gary Trent Jr', 'Larry Nance Jr',
                                                                                                                 'Duane Washington Jr', 'Scottie Pippen Jr', 'Vince Williams Jr', 'Ron Harper Jr'] else x)
        df['Player'] = df['Player'].apply(unidecode)

        df['Season'] = 2000+year
        df_overall = pd.concat([df_overall, df], ignore_index=False)

    df_overall = df_overall.reset_index(drop=True)
    return df_overall

In [103]:
ratings = get2kOverall()
print(f'Number of rows: {ratings.shape[0]}')
ratings.head()

Number of rows: 6239


,Player,Rating,Season
0,Kobe Bryant,97,2010
1,Dwyane Wade,96,2010
2,LeBron James,96,2010
3,Chris Paul,94,2010
4,Deron Williams,89,2010


### Merge data sources

In [148]:
def mergeData(stats, salaries, ratings, cap, rename_cols=False):
    
    
    df = pd.merge(stats, salaries, on=['Player', 'Season'], how='inner')
    no_salary = stats[~stats['Player'].isin(df['Player'])][['Player', 'Season']]

    df = pd.merge(df, ratings, on=['Player', 'Season'], how='inner')
    no_rating = stats[~stats['Player'].isin(df['Player'])][['Player', 'Season']]

    # Add the salary cap, and calculate the percentage of the cap that each player's salary represents
    df = pd.merge(df, cap, on='Season', how='left')
    df['Salary Cap %'] = (df['Salary'] / df['Cap Maximum']).round(3)
    df = df.round(3)

    # Get dummies for the position
    pos = pd.get_dummies(df['Pos'], prefix='Pos', dtype=int)
    df = pd.concat([df, pos], axis=1)
    df = df.drop(columns=['Pos'])

    df.fillna(0, inplace=True)

    # Rename columns for better understanding
    if rename_cols:
        col_names = {
            'G': 'Games', 'GS': 'Games Started', 'MP': 'Minutes Played', 
            'FG': 'Field Goals', 'FGA': 'Field Goals Attempted', 
            '3P': '3-Point Field Goals', '3PA': '3-Point Field Goals Attempted', 
            '2P': '2-Point Field Goals', '2PA': '2-Point Field Goals Attempted', 
            'FT': 'Free Throws', 'FTA': 'Free Throws Attempted', 
            'ORB': 'Offensive Rebounds', 'DRB': 'Defensive Rebounds', 'TRB': 'Total Rebounds',
            'AST': 'Assists', 'STL': 'Steals', 'BLK': 'Blocks',
            'TOV': 'Turnovers', 'PF': 'Personal Fouls', 'PTS': 'Points',
            'FG%': 'Field Goal %', '3P%': '3-Point Field Goal %', '2P%': '2-Point Field Goal %',
            'eFG%': 'Effective Field Goal %', 'FT%': 'Free Throw %',
            'Salary': 'Salary (M)', 'Cap Maximum': 'Cap Maximum (M)',
            'Pos_C': 'Center', 'Pos_PF': 'Power Forward', 'Pos_PG': 'Point Guard', 'Pos_SF': 'Small Forward', 'Pos_SG': 'Shooting Guard'
        }
        df = df.rename(columns=col_names)
        
    return df, no_salary, no_rating

In [150]:
data, no_salary, no_rating = mergeData(stats, salaries, ratings, cap)
print(f'Number of rows: {data.shape[0]}')
data.to_csv(os.path.join('..', 'data', 'processed', 'data.csv'), index=False)
data.head()

Number of rows: 2573


,Player,Age,G,GS,MP,FG,FGA,3P,3PA,2P,2PA,FT,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,FG%,3P%,2P%,eFG%,FT%,Season,Salary,Rating,Cap Maximum,Salary Cap %,Pos_C,Pos_PF,Pos_PG,Pos_SF,Pos_SG
0,Aaron Brooks,25,82,82,35.60,7.00,16.20,2.50,6.40,4.50,9.80,3.00,3.60,0.70,2.00,2.60,5.30,0.80,0.20,2.80,2.40,19.60,0.43,0.39,0.46,0.51,0.83,2010,1.12,71,58.04,0.02,0,0,1,0,0
1,Acie Law,25,52,2,37.20,5.50,11.30,0.70,2.30,4.80,9.00,6.30,8.00,0.40,1.90,2.40,4.00,2.00,0.10,2.40,2.60,17.80,0.49,0.30,0.53,0.52,0.79,2010,2.22,62,58.04,0.04,0,0,1,0,0
2,Adam Morrison,25,33,0,14.30,3.00,7.20,0.20,1.70,2.90,5.60,0.20,0.30,1.30,2.20,3.50,1.10,0.10,0.10,0.30,1.50,6.40,0.42,0.12,0.52,0.43,0.67,2010,5.26,55,58.04,0.09,0,0,0,1,0
3,Al Harrington,29,72,15,30.50,6.20,14.30,1.90,5.70,4.30,8.60,3.30,4.40,1.20,4.40,5.60,1.50,0.90,0.40,1.80,2.90,17.70,0.43,0.33,0.50,0.50,0.75,2010,10.03,75,58.04,0.17,0,1,0,0,0
4,Al Horford,23,92,92,70.40,11.90,22.10,0.10,0.10,11.80,21.90,5.00,6.10,5.40,13.50,18.90,4.10,1.40,2.80,2.90,6.00,28.80,0.54,1.00,0.54,0.54,0.82,2010,4.31,76,58.04,0.07,1,0,0,0,0
